In [1]:
!wget https://www.dropbox.com/s/4b9w456kxeg6yhh/dataset.zip?dl=0
!unzip /content/dataset.zip?dl=0

--2023-03-20 17:18:54--  https://www.dropbox.com/s/4b9w456kxeg6yhh/dataset.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/4b9w456kxeg6yhh/dataset.zip [following]
--2023-03-20 17:18:56--  https://www.dropbox.com/s/raw/4b9w456kxeg6yhh/dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca01c8d9d3b054885365a567397.dl.dropboxusercontent.com/cd/0/inline/B4leg08PY7Sc2Wl4LVx11NzDoKttEypSi4HdbWyrU6_5VL5s_cBq95xP_jGMH-xoD5CwIRBoB5GzwytZ48OuPjYO09LpCitO3D9NBItPVtFqA8ChAppwegyCuM2cDCil3p3ZZa2dEcvHOH7hAKwze-jv9AP5-bekpRoyOACjBMQvuQ/file# [following]
--2023-03-20 17:18:56--  https://uca01c8d9d3b054885365a567397.dl.dropboxusercontent.com/cd/0/inline/B4leg08PY7Sc2Wl4LVx11NzDoKttEypSi4HdbWyrU6_5VL5s_cBq95xP_jGMH-xoD5CwIRBoB5G

In [31]:
#Class to load the dataset images from drivce
import os
import cv2
import numpy as np


class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [32]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [33]:
from imutils import paths
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader




# Get list of image paths
image_paths = list(paths.list_images("/content/dataset"))

# Initialize SimplePreprocessor and SimpleDatasetLoader and load data and labels
print('[INFO]: Images loading....')
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(image_paths, verbose=500)

# Reshape from (3000, 32, 32, 3) to (3000, 32*32*3=3072)
data = data.reshape((data.shape[0], 3072))
# Print information about memory consumption
print('[INFO]: Features Matrix: {:.1f}MB'.format(float(data.nbytes / 1024*1000.0)))


[INFO]: Images loading....
[INFO]: Processed 500/1384
[INFO]: Processed 1000/1384
[INFO]: Features Matrix: 4152000.0MB


In [34]:
le = LabelEncoder()
lables = le.fit_transform(labels)

myset = set(lables)
print(myset)


{0, 1, 2, 3, 4, 5, 6}


In [36]:
from sklearn import metrics

In [39]:
(trainX, testX, trainY, testY ) = train_test_split(data, lables, test_size= 0.25, random_state=42)

model_knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
model_knn.fit(trainX, trainY)
y_prediction_knn = model_knn.predict(testX) 
score_knn = metrics.accuracy_score(y_prediction_knn, testY).round(4)
print("----------------------------------")
print('The accuracy of the KNN is: {}'.format(score_knn))
print("----------------------------------")
print(classification_report(testY, model_knn.predict(testX), target_names=le.classes_))

----------------------------------
The accuracy of the KNN is: 0.7803
----------------------------------
                      precision    recall  f1-score   support

Christian_ReachingUp       0.56      0.68      0.61        34
    Christian_bowing       0.42      0.26      0.32        19
  Christian_kneeling       0.71      0.60      0.65        25
         Muslim_Ioma       1.00      1.00      1.00        12
         Muslim_Qiam       0.93      0.99      0.96       142
         Muslim_Ruku       0.71      0.34      0.46        35
        Muslim_Sujud       0.67      0.78      0.73        79

            accuracy                           0.78       346
           macro avg       0.72      0.67      0.68       346
        weighted avg       0.77      0.78      0.77       346



In [41]:
from sklearn.linear_model import LogisticRegression
model_lr = LogisticRegression(solver='lbfgs', max_iter=500)
model_lr.fit(trainX, trainY) 
y_prediction_lr = model_lr.predict(testX)
score_lr = metrics.accuracy_score(y_prediction_lr, testY).round(4)
print("---------------------------------")
print('The accuracy of the LR is: {}'.format(score_lr))
print("---------------------------------")
print(classification_report(testY, model_knn.predict(testX), target_names=le.classes_))

---------------------------------
The accuracy of the LR is: 0.737
---------------------------------
                      precision    recall  f1-score   support

Christian_ReachingUp       0.56      0.68      0.61        34
    Christian_bowing       0.42      0.26      0.32        19
  Christian_kneeling       0.71      0.60      0.65        25
         Muslim_Ioma       1.00      1.00      1.00        12
         Muslim_Qiam       0.93      0.99      0.96       142
         Muslim_Ruku       0.71      0.34      0.46        35
        Muslim_Sujud       0.67      0.78      0.73        79

            accuracy                           0.78       346
           macro avg       0.72      0.67      0.68       346
        weighted avg       0.77      0.78      0.77       346



/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier(random_state=4)
model_dt.fit(trainX, trainY) 
y_prediction_dt = model_dt.predict(testX) 
score_dt = metrics.accuracy_score(y_prediction_dt, testY).round(4)
print("---------------------------------")
print('The accuracy of the DT is: {}'.format(score_dt))
print("---------------------------------")
print(classification_report(testY, model_knn.predict(testX), target_names=le.classes_))

---------------------------------
The accuracy of the DT is: 0.737
---------------------------------
                      precision    recall  f1-score   support

Christian_ReachingUp       0.56      0.68      0.61        34
    Christian_bowing       0.42      0.26      0.32        19
  Christian_kneeling       0.71      0.60      0.65        25
         Muslim_Ioma       1.00      1.00      1.00        12
         Muslim_Qiam       0.93      0.99      0.96       142
         Muslim_Ruku       0.71      0.34      0.46        35
        Muslim_Sujud       0.67      0.78      0.73        79

            accuracy                           0.78       346
           macro avg       0.72      0.67      0.68       346
        weighted avg       0.77      0.78      0.77       346



In [43]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=4)
model_rf.fit(trainX, trainY) 
y_prediction_rf = model_rf.predict(testX) 
score_rf = metrics.accuracy_score(y_prediction_rf, testY).round(4)
print("---------------------------------")
print('The accuracy of the DT is: {}'.format(score_rf))
print("---------------------------------")
print(classification_report(testY, model_knn.predict(testX), target_names=le.classes_))

---------------------------------
The accuracy of the DT is: 0.8035
---------------------------------
                      precision    recall  f1-score   support

Christian_ReachingUp       0.56      0.68      0.61        34
    Christian_bowing       0.42      0.26      0.32        19
  Christian_kneeling       0.71      0.60      0.65        25
         Muslim_Ioma       1.00      1.00      1.00        12
         Muslim_Qiam       0.93      0.99      0.96       142
         Muslim_Ruku       0.71      0.34      0.46        35
        Muslim_Sujud       0.67      0.78      0.73        79

            accuracy                           0.78       346
           macro avg       0.72      0.67      0.68       346
        weighted avg       0.77      0.78      0.77       346

